In [1]:
from matricesRalas import MatrizRala
from funciones_utiles import norma
import matplotlib.pyplot as plt
from typing import List, Tuple

Ahora tomemos el dataset de papers y realicemos un aálisis similar para este caso. Dado la gran cantidad $N$ de papers $(\approx 600.000)$, vamos a resolver el sistema solamente de forma iterativa, ya que utilizando Gauss-Jordan (es decir, considerando $\textbf{p*} = \textbf{p}_{t} = \textbf{p}_{t+1}$) se tiene una complejidad demasiado grande para utilizar el algoritmo en un tiempo razonable.

Recordando del ejercicio 3, se tienen las matrices $W, D  \in \mathbb{R}^{N \times N}$ tales que 

$$   
W_{ij} =
\begin{cases}
  1 & \text{si} \; j \;\text{cita a}\; i\\
  0 & \text{si no}
\end{cases}
$$

$$   
D_{ii} =
\begin{cases}
  \frac{1}{c_{i}} & \text{si} \; c_{i} \neq 0 \\
  0 & \text{si} \; c_{i} = 0
\end{cases}
$$

En donde $c_{i}$ es la cantidad de citas hechas por el $paper$ $i$:
$$
c_{i} = \sum_{k = 0}^{N} W_{ki}
$$

In [2]:
# Matrices papers
import csv

N = 0
citas_recibidas = {}
cantidad_citados = []
# citas_recibidas es un dict en donde para una clave i contiene una lista con los ids de los papers que citan al paper con id i
# cantidad_citados es una lista en donde cantidad_citados[i] contiene a la cantidad de citas realizadas por el paper con id i

# Se determina la cantidad de papers que hay
with open('papers/papers.csv', newline='') as csvfile:
    papers = csv.DictReader(csvfile)
    for row in papers:
        N += 1
        cantidad_citados.append(0)

# Se llena citas_recibidas y cantidad_citados
with open('papers/citas.csv', newline='') as csvfile:
    quotations = csv.DictReader(csvfile)
    for row in quotations:
        citador = int(row["from"])
        citado = int(row["to"])

        if citado not in citas_recibidas.keys():
          citas_recibidas[citado] = [citador]
        else:
          citas_recibidas[citado].append(citador) 
        
        cantidad_citados[citador] += 1
        
# Armamos W
W = MatrizRala(N,N)
for i in citas_recibidas.keys():
  for j in citas_recibidas[i]:
    W[i, j] = 1

# Armamos D
D = MatrizRala(N,N)
for i in range(N):
  if (cantidad_citados[i] != 0):
    D[i,i] = 1/cantidad_citados[i]

# Armamos el vector de 1's
unos = MatrizRala(N,1)
for i in range(N):
    unos[i,0] = 1

Queremos analizar la convergencia del sistema de forma iterativa. Para ellos iteramos siguiendo la ecuación $\textbf{p}_{t+1} = \frac{1-d}{N}*\mathbb{1} + dWD\textbf{p}_{t}$  hasta que $\|(\textbf{p}_{t+1} - \textbf{p}_{t})\| < \varepsilon$ para algún $\varepsilon > 0$, en este caso definimos $\varepsilon = 10^{-6}$ 

Ademas queremos graficar como se va modificando $\|(\textbf{p}_{t+1} - \textbf{p}_{t})\|$ entre cada iteracion, es decir queremos ver como se modifica el valor de $\|(\textbf{p}_{1} - \textbf{p}_{0})\|$ con respecto a $\|(\textbf{p}_{2} - \textbf{p}_{1})\|$ y asi sucesivamente.

In [3]:
# Método iterativo
import time

# Recordemos W, D y d
epsilon = 1e-6 # Arbitrario

d=0.85

# Construimos p_0
p_t = MatrizRala(N,1)
for i in range(N):
    p_t[i,0] = 1/N

# Lista de diferencias de normas.
# Para todo i (0 <= i <= |diferencias|) se tiene que, posicion diferencias[i] = norma(p_i+1 - p_i)
diferencias = []

constante = unos * ((1 - d) / N)
matrizMultiplo = d*W@D 

i = 0

while i == 0 or diferencias[i-1] > epsilon:
    if (i != 0):
        p_t = p_next
    
    inicio = time.time()
    
    # p_{t+1} = unos * (1-d)/N + d * W @ D @ p_t
    p_next = constante + matrizMultiplo @ p_t

    fin = time.time()
    print(fin-inicio)

    diferencias.append(norma(p_next-p_t))
    

    i += 1


# Gráfico de las diferencias
graf, ejes = plt.subplots()
plt.plot(diferencias, color="red")
ejes.set_xlabel("Cant. iteraciones (t)")
ejes.set_ylabel("Diferencia observada")
ejes.set_xticks([0,10,20,30,40,50,60,70,80,90,100,110,120,130])
plt.show()
plt.close()

8.864314079284668
4.170699834823608
6.074043273925781
9.871479749679565


Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x1069df340>>
Traceback (most recent call last):
  File "/Users/federicopeitti/miniconda3/envs/metodos/lib/python3.10/site-packages/ipykernel/ipkernel.py", line 770, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(
KeyboardInterrupt: 


Una vez que el sistema convergió, usamos el vector resultante para encontrar cuáles son los 10 papers de mayor impacto. A su vez también primero vemos cuáles son los papers más citados para comparar resultados.

In [ ]:
# SE HALLA EL TOP 10 DE PAPERS SEGUN LA CANTIDAD DE CITADOS

diezPopulares = dict() # La clave es el id del paper y el valor es la cantidad de citas que recibe
for paper, lista_citas in citas_recibidas.items():
    if len(diezPopulares) < 10:
        diezPopulares[paper] = len(lista_citas)
    else:
        citasMenosPopulart = 1e10
        paperMenosPopular: int = None
        for clave, valor in diezPopulares.items():
            if valor < citasMenosPopular:
                paperMenosPopular = clave
                citasMenosPopular = valor

        if len(lista_citas) > citasMenosPopular:
            diezPopulares.pop(paperMenosPopular)
            diezPopulares[paper] = len(lista_citas)

NameError: name 'citasMenosPopular' is not defined

In [ ]:
# SE HALLA EL TOP 10 DE PAPERS SEGUN EL ALGORITMO PAGERANK

diezImpacto = dict() # La clave es el id del paper y el valor es el correspondiente a ese paper en el vector p
for nro_paper, fila in p_next.filas.items():
    valorFinal: float = fila.raiz.valor[1]
    if len(diezImpacto) < 10:
        diezImpacto[nro_paper] = valorFinal
    else:
        menosImpacto: int = 1e10
        paperMenosImpacto: int = None
        for clave, valor in diezImpacto.items():
            if valor < menosImpacto:
                paperMenosImpacto = clave
                menosImpacto = valor

        if valorFinal > menosImpacto:
            diezImpacto.pop(paperMenosImpacto)
            diezImpacto[nro_paper] = valorFinal

In [ ]:

# Con esto, identificamos los papers para cada ranking

nombresAlgoritmo: List[Tuple[str, int, float]] = []
nombresCitasRecibidas: List[Tuple[str, int, int]] = []
with open('papers/papers.csv', newline='', encoding="utf-8") as csvfile:
    quotations = csv.DictReader(csvfile)
    for row in quotations:
        if int(row["id"]) in diezPopulares.keys():
            nombresCitasRecibidas.append((row["titulo"], int(row["id"]), diezPopulares[int(row["id"])]))
        if int(row["id"]) in diezImpacto.keys():
            nombresAlgoritmo.append((row["titulo"], int(row["id"]), diezImpacto[int(row["id"])]))

nombresAlgoritmo.sort(key=lambda x : x[2], reverse=True)

print("Papers con mayor impacto")
for nombre in nombresAlgoritmo: 
    print(f"Paper: {nombre[0]}, id: {nombre[1]}, proba:{nombre[2]}")

nombresCitasRecibidas.sort(key=lambda x : x[2], reverse=True)

print("\n")
print("Papers con más citas")
for nombre in nombresCitasRecibidas: 
    print(f"Paper: {nombre[0]}, id: {nombre[1]}, citas recibidas:{nombre[2]}")


Como puede observarse, según el algoritmo el paper con mayor impacto es "The art of computer programming, volume 2 (3rd ed.)". Luego, el paper más citado es "Introduction to algorithms". Esto indica que $PageRank$ no se guía únicamente por la cantidad de citas recibidas, si no que también por la calidad de las mismas.